In [2]:
#DEMO

import folium
import json
from urllib.request import urlopen

# URL zur Gemeinde Rifferswil (BFS-Nr 12)
url = "https://api3.geo.admin.ch/rest/services/api/MapServer/ch.swisstopo.swissboundaries3d-gemeinde-flaeche.fill/12?geometryFormat=geojson&sr=4326"

# GeoJSON herunterladen
with urlopen(url) as response:
    geojson_data = json.load(response)

# Mittelpunkt aus bbox berechnen
bbox = geojson_data["feature"]["bbox"]
center_lat = (bbox[1] + bbox[3]) / 2
center_lon = (bbox[0] + bbox[2]) / 2

# Karte erstellen
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles="CartoDB positron"
)

# GeoJSON hinzufügen
folium.GeoJson(
    geojson_data["feature"],
    name="Gemeinde Rifferswil",
    style_function=lambda x: {
        "fillColor": "#3186cc",
        "color": "black",
        "weight": 2,
        "fillOpacity": 0.5,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["gemname", "gde_nr", "jahr"],
        aliases=["Gemeinde", "BFS-Nr", "Jahr"]
    )
).add_to(m)

# Layer Control
folium.LayerControl().add_to(m)

# Speichern als HTML
file_path = "rifferswil_map.html"
m.save(file_path)

file_path


'rifferswil_map.html'

In [20]:
import json
import os
from urllib.request import urlopen
import folium

# -----------------------------
# 1. Gemeindezuweisungen laden
# -----------------------------
zuweisungen_url = "https://gebietsstammdaten.statistik.zh.ch/api/gemeindezuweisungen"

with urlopen(zuweisungen_url) as response:
    zuweisungen_data = json.load(response)

gemeinden = zuweisungen_data["gemeindezuweisungen"]

output_dir = "gemeinde_steckbriefe"
os.makedirs(output_dir, exist_ok=True)

# Logo-Dateiname
logo_filename = "../logo.svg"

# -----------------------------
# 2. Für jede Gemeinde Steckbrief erzeugen
# -----------------------------
for g in gemeinden:
    
    gemeinde_code = g["gemeinde_code"]
    gemeinde_name = g["gemeinde_name"]
    
    print(f"Erstelle Steckbrief für {gemeinde_name}...")

    geo_url = f"https://api3.geo.admin.ch/rest/services/api/MapServer/ch.swisstopo.swissboundaries3d-gemeinde-flaeche.fill/{gemeinde_code}?geometryFormat=geojson&sr=4326"

    try:
        with urlopen(geo_url) as response:
            geojson_data = json.load(response)
    except:
        print(f"⚠ Keine Geodaten für {gemeinde_name}")
        continue

    bbox = geojson_data["feature"]["bbox"]
    center_lat = (bbox[1] + bbox[3]) / 2
    center_lon = (bbox[0] + bbox[2]) / 2

    # -----------------------------
    # Karte erzeugen
    # -----------------------------
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles="CartoDB positron"
    )

    folium.GeoJson(
        geojson_data["feature"],
        style_function=lambda x: {
            "fillColor": "#003D7A",
            "color": "black",
            "weight": 2,
            "fillOpacity": 0.5,
        }
    ).add_to(m)

    safe_name = gemeinde_name.replace(" ", "_").replace("(", "").replace(")", "")

    # Unterordner "maps" erstellen, falls er noch nicht existiert
    maps_dir = os.path.join(output_dir, "maps")
    os.makedirs(maps_dir, exist_ok=True)

    # Karte speichern
    map_filename = f"{safe_name}_map.html"
    map_path = os.path.join(maps_dir, map_filename)
    m.save(map_path)

    # GeoJSON speichern
    geojson_filename = f"{safe_name}.geojson"
    geojson_path = os.path.join(maps_dir, geojson_filename)
    with open(geojson_path, "w", encoding="utf-8") as f:
        json.dump(geojson_data["feature"], f, ensure_ascii=False, indent=2)

    # -----------------------------
    # Steckbrief HTML
    # -----------------------------
    html_content = f"""
<!DOCTYPE html>
<html lang="de">
<head>
<meta charset="UTF-8">
<title>{gemeinde_name}</title>
<style>
body {{
    font-family: Arial, sans-serif;
    margin: 0;
    background-color: #f4f6f9;
}}

.header {{
    background-color: #0076bd;
    color: white;
    padding: 20px 40px;
    display: flex;
    align-items: center;
}}

.header img {{
    height: 60px;
    margin-right: 20px;
    filter: brightness(0) invert(1);
}}

.header h1 {{
    margin: 0;
}}

.content {{
    padding: 40px;
}}

.card {{
    background: white;
    padding: 25px;
    border-radius: 12px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.08);
    margin-bottom: 30px;
}}

.card h2 {{
    color: #0076bd;
    margin-top: 0;
}}

iframe {{
    width: 100%;
    height: 500px;
    border: none;
    border-radius: 12px;
}}

.download-btn {{
    background-color: #0076bd;
    color: white;
    padding: 12px 18px;
    border-radius: 6px;
    text-decoration: none;
    margin-right: 15px;
    display: inline-block;
}}

.download-btn:hover {{
    background-color: #0076bd;
}}
</style>
</head>

<body>

<div class="header">
    <img src="{logo_filename}" alt="Zürich Leu">
    <h1>{gemeinde_name}</h1>
</div>

<div class="content">

<div class="card">
    <p><strong>Gemeinde-Code:</strong> {g["gemeinde_code"]}</p>
    <p><strong>Bezirk:</strong> {g["bezirk_name"]} ({g["bezirk_code"]})</p>
    <p><strong>Raumplanungsregion:</strong> {g["raumplanungsregion_name"]} ({g["raumplanungsregion_code"]})</p>
</div>

<div class="card">
    <h2>Lage</h2>
    <iframe src="maps/{map_filename}"></iframe>
</div>

<div class="card">
    <h2>Downloads</h2>
    <a class="download-btn" href="maps/{map_filename}" download>Karte (HTML)</a>
    <a class="download-btn" href="maps/{geojson_filename}" download>GeoJSON</a>
</div>

</div>
        <footer>© Statistisches Amt des Kantons Zürich</footer>
</body>
</html>
"""

    file_path = os.path.join(output_dir, f"{safe_name}.html")

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(html_content)

print("✅ Alle Steckbriefe im Zürich-Design erstellt.")

# -----------------------------
# 3. Alles in ZIP packen
# -----------------------------

zip_filename = "gemeinde_steckbriefe.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            # relativer Pfad innerhalb von output_dir
            arcname = os.path.relpath(file_path, start=output_dir)
            zipf.write(file_path, arcname=arcname)




Erstelle Steckbrief für Aeugst am Albis...
Erstelle Steckbrief für Affoltern am Albis...
Erstelle Steckbrief für Bonstetten...
Erstelle Steckbrief für Hausen am Albis...
Erstelle Steckbrief für Hedingen...
Erstelle Steckbrief für Kappel am Albis...
Erstelle Steckbrief für Knonau...
Erstelle Steckbrief für Maschwanden...
Erstelle Steckbrief für Mettmenstetten...
Erstelle Steckbrief für Obfelden...
Erstelle Steckbrief für Ottenbach...
Erstelle Steckbrief für Rifferswil...
Erstelle Steckbrief für Stallikon...
Erstelle Steckbrief für Wettswil am Albis...
Erstelle Steckbrief für Benken (ZH)...
Erstelle Steckbrief für Berg am Irchel...
Erstelle Steckbrief für Buch am Irchel...
Erstelle Steckbrief für Dachsen...
Erstelle Steckbrief für Dorf...
Erstelle Steckbrief für Feuerthalen...
Erstelle Steckbrief für Flaach...
Erstelle Steckbrief für Flurlingen...
Erstelle Steckbrief für Henggart...
Erstelle Steckbrief für Kleinandelfingen...
Erstelle Steckbrief für Laufen-Uhwiesen...
Erstelle Steckbrief